**39. Combination Sum**

Given an array of distinct integers candidates and a target integer target, return a list of all unique combinations of candidates where the chosen numbers sum to target. You may return the combinations in any order.

The same number may be chosen from candidates an unlimited number of times. Two combinations are unique if the frequency of at least one of the chosen numbers is different.

The test cases are generated such that the number of unique combinations that sum up to target is less than 150 combinations for the given input.

https://leetcode.com/problems/combination-sum/

In [61]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

def make_linked_list(nums):
    linked_list = ListNode(val=nums[0])
    node = linked_list
    for n in nums[1:]:
        node.next = ListNode(val=n)
        node = node.next
    return linked_list

def print_linked_list(node):
    while node:
        print(node.val, end=" ")
        node = node.next
    print()

print_linked_list(make_linked_list([2,4,3]))
print_linked_list(make_linked_list([5,6,4]))

2 4 3 
5 6 4 


In [78]:
# dynamic programming
def combinationSum(candidates: list[int], target: int) -> list[list[int]]:
    candidates.sort()
    dp = [[] for _ in range(target+1)]
    for t in range(candidates[0], target+1):
        for c in candidates:
            if c > t:
                break
            elif c == t:
                dp[c].append([c])
            else:
                if dp[t-c]:
                    for l in dp[t-c]:
                        # only appending values that are smaller or equal
                        # than the last value in the candidate solution.
                        # this ensures that solutions are unique
                        if c <= l[-1]:
                            dp[t].append(l + [c])
    return dp[target]

# bfs
def combinationSum(candidates: list[int], target: int) -> list[list[int]]:
    from collections import deque
    candidates.sort()
    ans = []
    q = deque([([], 0, target)])
    while q:
        solution, index, new_target = q.popleft()
        for i in range(index, len(candidates)):
            if candidates[i] > new_target:
                break
            if candidates[i] == new_target:
                ans.append(solution + [candidates[i]])
            else:
                q.append((solution + [candidates[i]], i, new_target-candidates[i]))
    return ans


print(combinationSum(candidates = [2,3,6,7], target = 7)) # [[2,2,3],[7]]
print(combinationSum(candidates = [2,3,5], target = 8)) # [[2,2,2,2],[2,3,3],[3,5]]
print(combinationSum(candidates = [2], target = 1)) # []
print(combinationSum(candidates = [2], target = 2)) # [[2]]

[[7], [2, 2, 3]]
[[3, 5], [2, 3, 3], [2, 2, 2, 2]]
[]
[[2]]


**40. Combination Sum II**

Given a collection of candidate numbers (candidates) and a target number (target), find all unique combinations in candidates where the candidate numbers sum to target.

Each number in candidates may only be used once in the combination.

Note: The solution set must not contain duplicate combinations.

https://leetcode.com/problems/combination-sum-ii/

In [56]:
# bottom-up DP
def combinationSum2(candidates: list[int], target: int) -> list[list[int]]:
    candidates.sort()
    dp = [{()}] + [set() for _ in range(target)]
    for c in candidates:
        for t in range(target, c-1, -1):
            for tup in dp[t-c]:
                dp[t].add(tup + (c,))
    return dp[target]

# BFS
from collections import deque
def combinationSum2(candidates: list[int], target: int) -> list[list[int]]:
    candidates.sort()
    ans = []
    q = deque([(0, [], target)])
    while q:
        index, cand, new_target = q.pop()
        if new_target == 0:
            ans.append(cand)
        else:
            for i in range(index, len(candidates)):
                if i > index and candidates[i] == candidates[i-1]:
                    continue
                if candidates[i] > new_target:
                    break
                q.appendleft((i+1, cand+[candidates[i]], new_target-candidates[i]))
    return ans

# recursive
def combinationSum2(candidates: list[int], target: int) -> list[list[int]]:
    candidates.sort()
    ans = []
    def backtrack(lst, ind, tgt):
        # basecase
        if tgt == 0:
            ans.append(lst)
            return
        for i in range(ind, len(candidates)):
            if i > ind and candidates[i] == candidates[i-1]:
                continue
            if candidates[i] > tgt:
                break
            backtrack(lst+[candidates[i]], i+1, tgt-candidates[i])

    backtrack([], 0, target)
    return ans

print(combinationSum2(candidates = [10,1,2,7,6,1,5], target = 8)) # [[1,1,6], [1,2,5], [1,7], [2,6]]
print(combinationSum2(candidates = [2,5,2,1,2], target = 5)) # [[1,2,2], [5]]
print(combinationSum2(candidates = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], target = 27)) # []

[[1, 1, 6], [1, 2, 5], [1, 7], [2, 6]]
[[1, 2, 2], [5]]
[]


**41. First Missing Positive**

Given an unsorted integer array nums, return the smallest missing positive integer.

You must implement an algorithm that runs in O(n) time and uses constant extra space.

In [57]:
# obvious O(n) solution. O(n) space complexity
def firstMissingPositive(nums: list[int]) -> int:
    nums = set(nums)
    for n in range(1, len(nums) + 2):
        if n not in nums:
            return n

# another O(n) solution. Still O(n) space complexity
def firstMissingPositive(nums: list[int]) -> int:
    length = len(nums) + 1
    total = length * (1 + length) // 2 
    visited = set()
    for n in nums:
        if n < 1 or n >= length or n in visited:
            total -= length 
            visited.add(length)
            length -= 1
            while length in visited:
                length -= 1
        else:
            total -= n 
            visited.add(n)
    return total

# O(1) space complexity
def firstMissingPositive(nums: list[int]) -> int:
    for i in range(len(nums)):
        if nums[i] < 0:
            nums[i] = 0
    
    for n in nums:
        if 1 <= abs(n) <= len(nums):
            if not nums[abs(n)-1]:
                nums[abs(n)-1] = -len(nums) - 1
            else:
                nums[abs(n)-1] = -abs(nums[abs(n)-1])

    for i in range(len(nums)):
        if nums[i] >= 0:
            return i+1
    
    return len(nums) + 1 

        

print("Testcases - all unique numbers")
print(firstMissingPositive([1,2,0])) # 3
print(firstMissingPositive([3,4,-1,1])) # 2
print(firstMissingPositive([7,8,9,11,12])) # 1
print(firstMissingPositive([1,2,3,5,7])) # 4
print(firstMissingPositive([3,0,4,5])) # 1
print(firstMissingPositive([6,0,3,4,5,0])) # 1
print(firstMissingPositive([1])) # 2

print("Testcases - repeating numbers")
print(firstMissingPositive([1,1])) # 2
print(firstMissingPositive([1,1,1])) # 2
print(firstMissingPositive([1,1,1,3])) # 2
print(firstMissingPositive([1,2,2,4])) # 3
print(firstMissingPositive([2,2])) # 1

Testcases - all unique numbers
3
2
1
4
1
1
2
Testcases - repeating numbers
2
2
2
3
1


**45. Jump Game II**

You are given a 0-indexed array of integers nums of length n. You are initially positioned at nums[0].

Each element nums[i] represents the maximum length of a forward jump from index i. In other words, if you are at nums[i], you can jump to any nums[i + j] where:

* 0 <= j <= nums[i] and
* i + j < n

Return the minimum number of jumps to reach nums[n - 1]. The test cases are generated such that you can reach nums[n - 1].

In [2]:
def jump(nums: list[int]) -> int:
    if len(nums) > 1 and nums[0] >= len(nums)-1:
        return 1
    current = -1
    nxt = nums[0]
    jumps = 0
    for i in range(len(nums)-1):
        if i > current:
            current = nxt
            jumps += 1
        nxt = max(nxt, nums[i] + i)
        if nxt >= len(nums) - 1:
            return jumps + 1
    return jumps

**47. Permutations II**

Given a collection of numbers, nums, that might contain duplicates, return all possible unique permutations in any order.

https://leetcode.com/problems/permutations-ii/

In [10]:
# easy set way, slow
def permuteUnique(nums: list[int]) -> list[list[int]]:
    nums.sort()
    res = set()
    l = len(nums)
    indices = set(range(l))
    stack = [(tuple(), indices)]
    while stack:
        candidate, options = stack.pop()
        if len(candidate) == l:
            res.add(candidate)
        for op in options:
            options_copy = options.copy()
            options_copy.remove(op)
            stack.append((candidate + (nums[op],), options_copy))
    return res

# another way - faster
def permuteUnique(nums: list[int]) -> list[list[int]]:
    nums.sort()
    res = []
    l = len(nums)
    indices = set(range(l))
    stack = [([], indices)]
    while stack:
        candidate, options = stack.pop()
        if len(candidate) == l:
            res.append(candidate)
        for op in options:
            # if the chosen number is the same as the previous number, and the previous number has already been used
            if op > 0 and nums[op] == nums[op-1] and op-1 not in options:
                break
            options_copy = options.copy()
            options_copy.remove(op)
            stack.append((candidate + [nums[op]], options_copy))
    return res

# from leetcode
def permuteUnique(nums: list[int]) -> list[list[int]]:
    ans = [[]]
    for n in nums:
        new_ans = []
        for lst in ans:
            for i in range(len(lst) + 1):
                new_ans.append(lst[:i] + [n] + lst[i:])
                if i < len(lst) and lst[i] == n:
                    break
        ans = new_ans
    return ans

print(permuteUnique([1])) # [[1]]
print(permuteUnique([1,1,2])) # [[1,1,2], [1,2,1], [2,1,1]]
print(permuteUnique([1,1,2,2])) # [[2, 2, 1, 1], [2, 1, 2, 1], [1, 2, 2, 1], [2, 1, 1, 2], [1, 2, 1, 2], [1, 1, 2, 2]]
print(permuteUnique([1,1,1,2])) # [[1,1,1,2], [1,1,2,1], [1,2,1,1], [2,1,1,1]]
print(permuteUnique([1,2,3])) # [[1,2,3],[1,3,2],[2,1,3],[2,3,1],[3,1,2],[3,2,1]]

[[1]]
[[2, 1, 1], [1, 2, 1], [1, 1, 2]]
[[2, 2, 1, 1], [2, 1, 2, 1], [1, 2, 2, 1], [2, 1, 1, 2], [1, 2, 1, 2], [1, 1, 2, 2]]
[[2, 1, 1, 1], [1, 2, 1, 1], [1, 1, 2, 1], [1, 1, 1, 2]]
[[3, 2, 1], [2, 3, 1], [2, 1, 3], [3, 1, 2], [1, 3, 2], [1, 2, 3]]


**54. Spiral Matrix**

Given an m x n matrix, return all elements of the matrix in spiral order.

https://leetcode.com/problems/spiral-matrix/

In [31]:
# very long answer
def spiralOrder(matrix: list[list[int]]) -> list[int]:
    height = len(matrix)
    width = len(matrix[0])

    ans = matrix[0]
    remaining = height * width - width
    counter = 0
    h = 0
    w = width - 1
    decrease = 0
    moves = [1, -1, -1, 1]

    while remaining:
        if not counter % 2:
            decrease += 1
        index = counter%4
        if index % 2:
            for _ in range(width - decrease):
                w += moves[index]
                ans.append(matrix[h][w])
                remaining -= 1
        else:
            for _ in range(height - decrease):
                h += moves[index]
                ans.append(matrix[h][w])
                remaining -= 1
        counter += 1

    return ans

# another, using list(zip(*matrix))[::-1] to rotate the matrix counter clockwise
def spiralOrder(matrix: list[list[int]]) -> list[int]:
    ans = []
    while matrix:
        ans += list(matrix.pop(0))
        matrix = [*zip(*matrix)][::-1]
    return ans

print(spiralOrder([[1,2],[3,4],[5,6]])) # [1,2,4,6,5,3]
print(spiralOrder([[1,2,3],[4,5,6],[7,8,9]])) # [1,2,3,6,9,8,7,4,5]
print(spiralOrder([[1,2,3,4],[5,6,7,8],[9,10,11,12]])) # [1,2,3,4,8,12,11,10,9,5,6,7]

[1, 2, 4, 6, 5, 3]
[1, 2, 3, 6, 9, 8, 7, 4, 5]
[1, 2, 3, 4, 8, 12, 11, 10, 9, 5, 6, 7]


**57. Insert Interval**

You are given an array of non-overlapping intervals intervals where intervals[i] = [starti, endi] represent the start and the end of the ith interval and intervals is sorted in ascending order by starti. You are also given an interval newInterval = [start, end] that represents the start and end of another interval.

Insert newInterval into intervals such that intervals is still sorted in ascending order by starti and intervals still does not have any overlapping intervals (merge overlapping intervals if necessary).

Return intervals after the insertion.

https://leetcode.com/problems/insert-interval/

In [59]:
# longwinded way, optimised using binary search
def insert(intervals: list[list[int]], newInterval: list[int]) -> list[list[int]]:

    def binary(start, end, ind):
        while start <= end:
            mid = (start + end) // 2
            if intervals[mid][ind] >= newInterval[ind]:
                end = mid-1
            else:
                start = mid+1
        return start
        
    if not intervals:
        return [newInterval]
    
    start = 0 if newInterval[0] <= intervals[0][0] else binary(0, len(intervals)-1, 0)
    end = len(intervals) if newInterval[1] >= intervals[-1][1] else binary(start, len(intervals)-1, 1)

    out = intervals[:start]
    if out and out[-1][1] >= newInterval[0]:
        out[-1][1] = max(newInterval[1], out[-1][1])
    else:
        out.append(newInterval)
    if end < len(intervals) and out[-1][1] >= intervals[end][0]:
        out[-1][1] = intervals[end][1]
        return out + intervals[end+1:]
    else:
        return out + intervals[end:]

# simple linear - slower
def insert(intervals: list[list[int]], newInterval: list[int]) -> list[list[int]]:
    i = 0
    ans = []
    while i < len(intervals) and intervals[i][0] < newInterval[0]:
        ans.append(intervals[i])
        i += 1
    if ans and ans[-1][1] >= newInterval[0]:
        ans[-1][1] = max(newInterval[1], ans[-1][1])
    else:
        ans.append(newInterval)
    while i < len(intervals) and intervals[i][1] <= newInterval[1]:
        i += 1
    if i < len(intervals) and ans[-1][1] >= intervals[i][0]:
        ans[-1][1] = intervals[i][1]
        return ans + intervals[i+1:]
    else:
        return ans + intervals[i:]



print(insert(intervals = [[1,5]], newInterval = [2,3])) # [[1,5]]
print(insert(intervals = [], newInterval = [5,7])) # [[5,7]]
print(insert(intervals = [[1,3],[6,9]], newInterval = [2,5])) # [[1,5],[6,9]]
print(insert(intervals = [[1,2],[3,5],[6,7],[8,10],[12,16]], newInterval = [4,8])) # [[1,2],[3,10],[12,16]]
print(insert(intervals = [[0,4],[7,12]], newInterval = [0,5])) # [[0,5],[7,12]]

[[1, 5]]
[[5, 7]]
[[1, 5], [6, 9]]
[[1, 2], [3, 10], [12, 16]]
[[0, 5], [7, 12]]


**58. Length of Last Word**

Given a string s consisting of words and spaces, return the length of the last word in the string.

A word is a maximal substring consisting of non-space characters only.

In [ ]:
# one liner
def lengthOfLastWord(s: str) -> int:
    return len(s.rstrip().split(" ")[-1])

# another solution
def lengthOfLastWord(s: str) -> int:
    word = None
    for i in range(len(s)-1, -1, -1):
        if s[i] == " ":
            if word:
                return word - i
        elif not word:
            word = i
    return word + 1

**59. Spiral Matrix II**

Given a positive integer n, generate an n x n matrix filled with elements from 1 to n^2 in spiral order.

https://leetcode.com/problems/spiral-matrix-ii/

In [47]:
def generateMatrix(n: int) -> list[list[int]]:
    matrix = [[0 for _ in range(n)] for _ in range(n)]
    number = 1
    next = [0,0]
    chosen = 0
    while True:
        h,w = next
        matrix[h][w] = number
        neighbours = [[h,w+1],[h+1,w],[h,w-1],[h-1,w]]
        hh,ww = neighbours[chosen%4]
        if hh < 0 or ww < 0 or hh == n or ww == n or matrix[hh][ww]:
            chosen += 1
            hh,ww = neighbours[chosen%4]
            if hh < 0 or ww < 0 or hh == n or ww == n or matrix[hh][ww]:
                return matrix
        next = [hh,ww]
        number += 1

print(generateMatrix(3)) # [[1,2,3],[8,9,4],[7,6,5]]
print(generateMatrix(1)) # [[1]]
print(generateMatrix(4)) # [[1,2,3,4],[12,13,14,5],[11,16,15,6],[10,9,8,7]]

[[1, 2, 3], [8, 9, 4], [7, 6, 5]]
[[1]]
[[1, 2, 3, 4], [12, 13, 14, 5], [11, 16, 15, 6], [10, 9, 8, 7]]


**61. Rotate List**

Given the head of a linked list, rotate the list to the right by k places.

https://leetcode.com/problems/rotate-list/

In [129]:
def rotateRight(head, k: int):
    if not head or not head.next or not k:
        return head
    counter = 1
    node = head.next
    while node.next:
        node = node.next
        counter += 1
    length = counter + 1
    k %= length
    node.next = head
    node = head
    counter = 0
    while counter < (length - k - 1):
        node = node.next
        counter += 1
    out = node.next
    node.next = None
    return out

print_linked_list(rotateRight(make_linked_list([1,2,3,4,5]), 2)) # 4 5 1 2 3 
print_linked_list(rotateRight(make_linked_list([0,1,2]), 4)) # 2 0 1 
print_linked_list(rotateRight(make_linked_list([1,2]), 0)) # 1 2 
print_linked_list(rotateRight(make_linked_list([1,2]), 1)) # 2 1

4 5 1 2 3 
2 0 1 
1 2 
2 1 


**63. Unique Paths II**

You are given an m x n integer array grid. There is a robot initially located at the top-left corner (i.e., grid[0][0]). The robot tries to move to the bottom-right corner (i.e., grid[m - 1][n - 1]). The robot can only move either down or right at any point in time.

An obstacle and space are marked as 1 or 0 respectively in grid. A path that the robot takes cannot include any square that is an obstacle.

Return the number of possible unique paths that the robot can take to reach the bottom-right corner.

The testcases are generated so that the answer will be less than or equal to 2 * 10^9.

https://leetcode.com/problems/unique-paths-ii/

In [5]:
def uniquePathsWithObstacles(obstacleGrid: list[list[int]]) -> int:
    height = len(obstacleGrid)
    width = len(obstacleGrid[0])
    if obstacleGrid[height-1][width-1] == 1:
        return 0
    dp = [[0 for _ in range(width+1)] for _ in range(height+1)]
    dp[height-1][width] = 1

    for h in range(height-1, -1, -1):
        for w in range(width-1, -1, -1):
            if obstacleGrid[h][w] == 0:
                dp[h][w] = dp[h+1][w] + dp[h][w+1]
    
    return dp[0][0]


print(uniquePathsWithObstacles([[0,0,0],[0,1,0],[0,0,0]])) # 2
print(uniquePathsWithObstacles([[0,1],[0,0]])) # 1

2
1


**67. Add Binary**

Given two binary strings a and b, return their sum as a binary string.

https://leetcode.com/problems/add-binary/

In [163]:
def addBinary(a: str, b: str) -> str:
    ans = []
    if len(a) != len(b):
        if len(a) > len(b):
            b = b.zfill(len(a))
        else:
            a = a.zfill(len(b))
    a = a[::-1]
    b = b[::-1]
    carry = False
    for i in range(len(a)):
        if a[i] != b[i]:
            if carry:
                ans.append("0")
                carry = True
            else:
                ans.append("1")
        elif a[i] == "1":
            if carry:
                ans.append("1")
            else:
                ans.append("0")
                carry = True
        else:
            if carry:
                ans.append("1")
                carry = False
            else:
                ans.append("0")
    if carry:
        ans.append("1")
    return "".join(ans[::-1])


print(addBinary(a = "11", b = "1")) # "100"
print(addBinary(a = "1", b = "11")) # "100"
print(addBinary(a = "1010", b = "1011")) # "10101"

100
100
10101


**71. Simplify Path**

Given a string path, which is an absolute path (starting with a slash '/') to a file or directory in a Unix-style file system, convert it to the simplified canonical path.

In a Unix-style file system, a period '.' refers to the current directory, a double period '..' refers to the directory up a level, and any multiple consecutive slashes (i.e. '//') are treated as a single slash '/'. For this problem, any other format of periods such as '...' are treated as file/directory names.

The canonical path should have the following format:

* The path starts with a single slash '/'.
* Any two directories are separated by a single slash '/'.
* The path does not end with a trailing '/'.
* The path only contains the directories on the path from the root directory to the target file or directory (i.e., no period '.' or double period '..')

Return the simplified canonical path.

https://leetcode.com/problems/simplify-path/

In [74]:
def simplifyPath(path: str) -> str:
    path += "/"
    ans = []
    current = []
    dots = 0
    for c in path:
        if c == "/":
            if dots == len(current) == 2:
                if ans:
                    ans.pop()
            elif current and current != ["."]:
                ans.append(current)
            dots = 0
            current = []
        else:
            if c == ".":
                dots += 1
            current.append(c)
    return "/" + "/".join(["".join(word) for word in ans])
        

print(simplifyPath("/home/")) # "/home"
print(simplifyPath("/../")) # "/"
print(simplifyPath("/home//foo/")) # "/home/foo"
print(simplifyPath("/a//b////c/d//././/..")) # "/a/b/c"
print(simplifyPath("/..hidden")) # "/..hidden"
print(simplifyPath("/hello../world")) # "/hello../world"
print(simplifyPath("/...")) # "/..."
print(simplifyPath("/hello./world/")) # "/hello./world"
print(simplifyPath("/../..ga/b/.f..d/..../e.baaeeh./.a")) # "/..ga/b/.f..d/..../e.baaeeh./.a"

/home
/
/home/foo
/a/b/c
/..hidden
/hello../world
/...
/hello./world
/..ga/b/.f..d/..../e.baaeeh./.a


**83. Remove Duplicates from Sorted List**

Given the head of a sorted linked list, delete all duplicates such that each element appears only once. Return the linked list sorted as well.

In [5]:
def deleteDuplicates(head):
    if not head or not head.next:
        return head
    node = head 
    while node.next:
        if node.val == node.next.val:
            node.next = node.next.next
        else:
            node = node.next
    return head 

**216. Combination Sum III**

Find all valid combinations of k numbers that sum up to n such that the following conditions are true:

Only numbers 1 through 9 are used.
Each number is used at most once.
Return a list of all possible valid combinations. The list must not contain the same combination twice, and the combinations may be returned in any order.

https://leetcode.com/problems/combination-sum-iii/

In [73]:
# dfs
def combinationSum3(k: int, n: int) -> list[list[int]]:
    if n < k or n > (9 * k):
        return []
    ans = []
    s = [([], 1, n)]
    while s:
        lst, num, tgt = s.pop()
        if len(lst) == k:
            if tgt == 0:
                ans.append(lst)
        else:
            for i in range(num, 10):
                s.append((lst + [i], i+1, tgt-i))
    return ans

# recursive
def combinationSum3(k: int, n: int) -> list[list[int]]:
    if n < k or n > (9 * k):
        return []
    ans = []
    
    def dfs(lst, num, tgt):
        if len(lst) == k:
            if tgt == 0:
                ans.append(lst)
            return
        for i in range(num, 10):
            dfs(lst + [i], i+1, tgt-i)

    dfs([], 1, n)

    return ans

print(combinationSum3(k = 3, n = 7))
print(combinationSum3(k = 3, n = 9))
print(combinationSum3(k = 4, n = 1))

[[1, 2, 4]]
[[1, 2, 6], [1, 3, 5], [2, 3, 4]]
[]


**516. Longest Palindromic Subsequence**

Given a string s, find the longest palindromic subsequence's length in s.

A subsequence is a sequence that can be derived from another sequence by deleting some or no elements without changing the order of the remaining elements.

https://leetcode.com/problems/longest-palindromic-subsequence/

In [43]:
# basically longest common subsequence. A little slow
def longestPalindromeSubseq(s: str) -> int:
    l = len(s)
    dp = [[0] * (l + 1) for _ in range(l + 1)]
    for i in range(l-1, -1, -1):
        for j in range(l-1, -1, -1):
            if s[i] == s[l-j-1]:
                dp[i][j] = dp[i+1][j+1] + 1
            else:
                dp[i][j] = max(dp[i+1][j], dp[i][j+1]) 
    return dp[0][0]

print(longestPalindromeSubseq("bbbab")) # 4
print(longestPalindromeSubseq("cbbd")) # 2
print(longestPalindromeSubseq("bdbbcabac")) # 5

4
2
5


**946. Validate Stack Sequences**

Given two integer arrays pushed and popped each with distinct values, return true if this could have been the result of a sequence of push and pop operations on an initially empty stack, or false otherwise.

In [30]:
# slightly longwinded
def validateStackSequences(pushed: list[int], popped: list[int]) -> bool:
    push_i = 0
    pop_i = 0
    stack = []
    while push_i < len(pushed) or pop_i < len(popped):
        try:
            if not stack or stack[-1] != popped[pop_i]:
                stack.append(pushed[push_i])
                push_i += 1
            elif stack[-1] == popped[pop_i]:
                pop_i += 1
                stack.pop()
        except:
            return False
    return True

# more compact
def validateStackSequences(pushed: list[int], popped: list[int]) -> bool:
    pop_i = 0
    stack = []
    for push in pushed:
        stack.append(push) 
        while stack and stack[-1] == popped[pop_i]:
            stack.pop()
            pop_i += 1
    return not stack


print(validateStackSequences(pushed = [1,2,3,4,5], popped = [4,5,3,2,1])) # True
print(validateStackSequences(pushed = [1,2,3,4,5], popped = [4,3,5,1,2])) # False

True
False


**1143. Longest Common Subsequence**

Given two strings text1 and text2, return the length of their longest common subsequence. If there is no common subsequence, return 0.

A subsequence of a string is a new string generated from the original string with some characters (can be none) deleted without changing the relative order of the remaining characters.

For example, "ace" is a subsequence of "abcde".
A common subsequence of two strings is a subsequence that is common to both strings.

https://leetcode.com/problems/longest-common-subsequence/

In [57]:
# standard DP
def longestCommonSubsequence(text1: str, text2: str) -> int:
    dp = [[0] * (len(text1) + 1) for _ in range(len(text2) + 1)]
    for i in range(len(text2)-1, -1, -1):
        for j in range(len(text1)-1, -1, -1):
            if text2[i] == text1[j]:
                dp[i][j] = dp[i+1][j+1] + 1
            else:
                dp[i][j] = max(dp[i][j+1], dp[i+1][j])
    return dp[0][0]

# very fast solution - similar to Longest Increasing Subsequence using Patience card game algorithm
def longestCommonSubsequence(text1: str, text2: str) -> int:
    dp = []
    d = dict()
    for i in range(len(text2)):
        if text2[i] in d:
            d[text2[i]].append(i)
        else:
            d[text2[i]] = [i]
    for c in text1:
        if c in d:
            for i in d[c][::-1]:
                l, r = 0, len(dp)-1
                while l <= r:
                    m = (l + r)//2
                    if dp[m] >= i:
                        r = m - 1
                    else:
                        l = m + 1
                if l == len(dp):
                    dp.append(i)
                else:
                    dp[l] = i
    return len(dp)

# print(longestCommonSubsequence(text1 = "abcde", text2 = "ace" )) # 3
# print(longestCommonSubsequence(text1 = "abc", text2 = "abc")) # 3
# print(longestCommonSubsequence(text1 = "abc", text2 = "def")) # 0
# print(longestCommonSubsequence(text1 = "bl", text2 = "yby")) # 1
print(longestCommonSubsequence(text1 = "abcba", text2 = "abcbcba")) # 5
# print(longestCommonSubsequence(text1 = "bsbininm", text2 = "jmjkbkjkv")) # 1
# print(longestCommonSubsequence(text1 = "bmvcnwrmxcfcxabkxcvgbozmpspsbenazglyxkpibgzq", text2 = "bmpmlstotylonkvmhqjyxmnqzctonqtobahcrcbibgzgx")) # 13

5


**1431. Kids With the Greatest Number of Candies**

There are n kids with candies. You are given an integer array candies, where each candies[i] represents the number of candies the ith kid has, and an integer extraCandies, denoting the number of extra candies that you have.

Return a boolean array result of length n, where result[i] is true if, after giving the ith kid all the extraCandies, they will have the greatest number of candies among all the kids, or false otherwise.

Note that multiple kids can have the greatest number of candies.

https://leetcode.com/problems/kids-with-the-greatest-number-of-candies/

In [11]:
def kidsWithCandies(candies: list[int], extraCandies: int) -> list[bool]:
    max_candies = max(candies)
    return [c+extraCandies >= max_candies for c in candies]

print(kidsWithCandies(candies = [2,3,5,1,3], extraCandies = 3)) # [true,true,true,false,true] 
print(kidsWithCandies(candies = [4,2,1,1,2], extraCandies = 1)) # [true,false,false,false,false] 
print(kidsWithCandies(candies = [12,1,12], extraCandies = 10)) # [true,false,true]


[True, True, True, False, True]
[True, False, False, False, False]
[True, False, True]


**1639. Number of Ways to Form a Target String Given a Dictionary**

You are given a list of strings of the same length words and a string target.

Your task is to form target using the given words under the following rules:

target should be formed from left to right.
To form the ith character (0-indexed) of target, you can choose the kth character of the jth string in words if target[i] = words[j][k].
Once you use the kth character of the jth string of words, you can no longer use the xth character of any string in words where x <= k. In other words, all characters to the left of or at index k become unusuable for every string.
Repeat the process until you form the string target.
Notice that you can use multiple characters from the same string in words provided the conditions above are met.

Return the number of ways to form target from words. Since the answer may be too large, return it modulo 10^9 + 7.

https://leetcode.com/problems/number-of-ways-to-form-a-target-string-given-a-dictionary/

In [9]:
# first solution
def numWays(words: list[str], target: str) -> int:
    if len(target) > len(words[0]):
        return 0
    dp = [[0 for _ in range(len(words[0])+1)] for _ in range(len(target))]
    dp += [[1 for _ in range(len(words[0])+1)]]
    for ti in range(len(target)-1, -1, -1):
        for wi in range(len(words[0])-(len(target)-ti), ti-1, -1):
            count = 0
            for w in words:
                if w[wi] == target[ti]:
                    count += 1
            dp[ti][wi] = (count * dp[ti+1][wi+1] + dp[ti][wi+1]) % (10**9+7)
        if dp[ti][ti] == 0:
            return 0
    
    return dp[0][0]

# slightly optimised - makes note of the first non-zero cell
def numWays(words: list[str], target: str) -> int:
    if len(target) > len(words[0]):
        return 0
    dp = [[0 for _ in range(len(words[0])+1)] for _ in range(len(target))]
    dp += [[1 for _ in range(len(words[0])+1)]]
    start = len(words[0])-1
    for ti in range(len(target)-1, -1, -1):
        for wi in range(start, ti-1, -1):
            count = 0
            for w in words:
                if w[wi] == target[ti]:
                    count += 1
            dp[ti][wi] = (count * dp[ti+1][wi+1] + dp[ti][wi+1]) % (10**9+7)
            if dp[ti][wi] == 0:
                start = wi - 1
        if dp[ti][ti] == 0:
            return 0
        start -= 1
    
    return dp[0][0]

print(numWays(words = ["acca","bbbb","caca"], target = "aba")) # 6
print(numWays(words = ["abba","baab"], target = "bab")) # 4

6
4


**1857. Largest Color Value in a Directed Graph**

There is a directed graph of n colored nodes and m edges. The nodes are numbered from 0 to n - 1.

You are given a string colors where colors[i] is a lowercase English letter representing the color of the ith node in this graph (0-indexed). You are also given a 2D array edges where edges[j] = [aj, bj] indicates that there is a directed edge from node aj to node bj.

A valid path in the graph is a sequence of nodes x1 -> x2 -> x3 -> ... -> xk such that there is a directed edge from xi to xi+1 for every 1 <= i < k. The color value of the path is the number of nodes that are colored the most frequently occurring color along that path.

Return the largest color value of any valid path in the given graph, or -1 if the graph contains a cycle.

https://leetcode.com/problems/largest-color-value-in-a-directed-graph/

In [54]:
# times out
def largestPathValue(colors: str, edges: list[list[int]]) -> int:
    if not edges:
        return 1
    
    graph = {}
    from_nodes = set()
    to_nodes = set()
    for k, v in edges:
        from_nodes.add(k)
        to_nodes.add(v)
        if k in graph:
            graph[k].append(v)
        else:
            graph[k] = [v]

    from_nodes -= to_nodes

    if not from_nodes:
        return -1
    
    largest_color_value = 0

    def bfs(start):
        s = [[start, dict(), set()]]
        while s:
            i, color_dic, visited = s.pop()
            visited.add(i)
            color_dic[colors[i]] = color_dic.get(colors[i], 0) + 1
            nonlocal largest_color_value
            largest_color_value = max(largest_color_value, color_dic[colors[i]])
            if i in graph:
                for n in graph[i]:
                    if n in visited:
                        return False
                    else:
                        s.append([n, color_dic.copy(), visited.copy()])
        return True

    for node in from_nodes:
        if not bfs(node):
            return -1

    return largest_color_value

print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4]])) # 3
print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4],[4,1]])) # 3
print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4],[4,2]])) # -1
print(largestPathValue(colors = "bbbhb", edges = [[0,2],[3,0],[1,3],[4,1]])) # 4
print(largestPathValue(colors = "a", edges = [[0,0]])) # -1
print(largestPathValue(colors = "g", edges = [])) # -1

3
3
-1
4
-1
1


In [72]:
# uses topological sort - very fast!
def largestPathValue(colors: str, edges: list[list[int]]) -> int:

    # no edges = max path is 1
    if not edges:
        return 1
    
    digraph = {} # storing the graph
    indegrees = {} # storing number of indegrees

    for k, v in edges:
        if k in digraph:
            digraph[k].append(v) # appending an edge
        else:
            digraph[k] = [v] # adding an edge to a node without any edges
        if v not in digraph:
            # if an edge leads to a node without any outcoming edges, we add that node to the dictionary
            # this step is needed to store nodes with no outcoming edges
            digraph[v] = []
        # counting indegrees
        indegrees[v] = indegrees.get(v, 0) + 1

    topological_sort = []
    # digraph contains all nodes as keys. indegrees contains only the nodes with incoming edges
    no_incoming_edges = list(digraph.keys() - indegrees.keys()) 

    # populating topological sort
    while no_incoming_edges:
        node = no_incoming_edges.pop()
        topological_sort.append(node)
        # iterating through the outcoming edges of the popped node
        for n in digraph[node]:
            # decrementing their indegrees
            indegrees[n] -= 1
            # if the popped node was their last remaining incoming edge, we add them to the topological sort
            if indegrees[n] == 0:
                no_incoming_edges.append(n)

    # this means there is a loop
    if len(digraph) != len(topological_sort):
        return -1
    
    largest_color_value = 0

    # maintaining a color dictionary for each node
    colour_dicts = {k:dict() for k in digraph.keys()}

    # going backwards in the topological sort
    for i in range(len(topological_sort)-1, -1, -1):
        index = topological_sort[i]

        # iterating through outgoing edges
        for n in digraph[index]:
            for color in colour_dicts[n]:
                # choosing maximum possible number of colours available to any child
                colour_dicts[index][color] = max(colour_dicts[index].get(color, 0), colour_dicts[n][color])
        # incrementing the node colour counter
        colour_dicts[index][colors[index]] = colour_dicts[index].get(colors[index], 0) + 1
        # potentially updating the largest_color_value counter
        largest_color_value = max(largest_color_value, colour_dicts[index][colors[index]])

    return largest_color_value


print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4]])) # 3
print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4],[4,1]])) # 3
print(largestPathValue(colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4],[4,2]])) # -1
print(largestPathValue(colors = "bbbhb", edges = [[0,2],[3,0],[1,3],[4,1]])) # 4
print(largestPathValue(colors = "a", edges = [[0,0]])) # -1
print(largestPathValue(colors = "g", edges = [])) # -1

3
3
-1
4
-1
1


**2218. Maximum Value of K Coins From Piles**
 
There are n piles of coins on a table. Each pile consists of a positive number of coins of assorted denominations.

In one move, you can choose any coin on top of any pile, remove it, and add it to your wallet.

Given a list piles, where piles[i] is a list of integers denoting the composition of the ith pile from top to bottom, and a positive integer k, return the maximum total value of coins you can have in your wallet if you choose exactly k coins optimally.

https://leetcode.com/problems/maximum-value-of-k-coins-from-piles/

In [98]:
# Slow DP. Same O(k*s), where s is the total number of coins in all piles,
# as the best solutions. Other solutions can be faster because they don't use the auxillary 
# weights and values arrays. 
def maxValueOfCoins(piles: list[list[int]], k: int) -> int:
    weights = [0]
    values = [0]
    for p in piles:
        added = 0
        for i in range(len(p)):
            if i < k:
                weights.append(i+1)
                added += p[i]
                values.append(added)

    dp = [[0] * (k+1) for _ in range(len(weights))]

    for i in range(1, len(weights)):
        for w in range(1, k+1):
            if weights[i] <= w:
                dp[i][w] = max(dp[i-1][w], dp[i-weights[i]][w-weights[i]] + values[i])
            else:
                dp[i][w] = dp[i-1][w]

    # ####
    # print(weights, values)
    # for r in dp:
    #     print(*r, sep="\t")
    # ####

    return dp[len(weights)-1][k]


print(maxValueOfCoins(piles = [[1,100,3],[7,8,9]], k = 2)) # 101
print(maxValueOfCoins(piles = [[100],[100],[100],[100],[100],[100],[1,1,1,1,1,1,700]], k = 7)) # 706
print(maxValueOfCoins(piles = [[100,100,100],[50]], k = 4)) # 350
print(maxValueOfCoins(piles = [[37,88],[51,64,65,20,95,30,26],[9,62,20],[44]], k = 9)) # 494

101
706
350
494


**2390. Removing Stars From a String**

You are given a string s, which contains stars *.

In one operation, you can:

Choose a star in s.
Remove the closest non-star character to its left, as well as remove the star itself.
Return the string after all stars have been removed.

Note:

The input will be generated such that the operation is always possible.
It can be shown that the resulting string will always be unique.

https://leetcode.com/problems/removing-stars-from-a-string/

In [65]:
def removeStars(s: str) -> str:
    stack = []
    for c in s:
        if c == "*":
            stack.pop()
        else:
            stack.append(c)
    return "".join(stack)

print(removeStars(s = "leet**cod*e")) # lecoe

lecoe
